### Intro

Backtracking is an understudied topic in my opinion when it comes to LeetCode interview questions. If you aren't comfortable with backtracking I highly recommend spending some time learning the algorithm as it can be used to solve a wide variety of questions as I will demonstrate in the examples section. At its core, backtracking is simply a special recursive form of Depth First Search (DFS). Like DFS, backtracking explores a path as deeply as possible before continuing along a new path. Unlike DFS however, backtracking keeps track of all values encountered along the current path as a candidate solution, typically in some kind of container data structure. Also unlike DFS, as soon as the algorithm determines the current candidate cannot lead to a valid solution it will "backtrack" by abandoning the last encountered value along the current path (e.g. popping the last value from a list) and then vist the next path.

Backtracking is used to solve Constraint Satisfaction Problems (CSPs), which are questions that define a set of objects and ask you to manipulate the objects in a way that satisfies a set of constraints. A classic backtracking example is [N-Queens](https://leetcode.com/problems/n-queens/) which asks you to place `n` queens on an `n x n` chessboard such that no two queens attack each other. Your object in this case is the chessboard, your constraint is that no two queens can attack each other, and a container object you could use to record a solution is a list with tuple coordinate pairs of successfully placed queens (e.g. queens placed on (0,0) and (1,2): `[(0,0), (1,2)]`). A backtracking algorithm would solve this problem by attempting to place queens onto the chessboard until it has successfully place `n` queens. If it is unable to place a queen before reaching `n` placements, the algorithm has reached a deadend along the current path and must backtrack by removing the last placed queen (popping the last tuple coordinate in our list) and attempting to place it in a different spot.


### Template

In [ ]:
def backtrack(candidate):
    if find_solution(candidate):               # recursive base case, current candidate solution satifies the CSP
        output(candidate)
        return
    
    for next_candidate in list_of_candidates:  # iterate over all possible candidates (e.g. all possible next letters or next board spaces)
        if is_valid(next_candidate):           # filter for valid candidates that don't violate the CSP
            place(next_candidate)              # try this partial candidate solution
            backtrack(next_candidate)          # given the candidate, explore further
            remove(next_candidate)             # if backtrack() returns unsuccessfully, backtrack by updating last candidate

If you want to terminate the recursion as soon as you find one successful solution to the CSP, you can update the template by using the code below. For example, for the N-Queens problem if we just want to determine whether or not there exists a board state that satisfies the constraints you can use the below template. If however we get a modification of the problem where we're asked to find how many ways can we uniquely place queens onto the board and still satisfy the constraints, then you can't terminate the recursion early and must use the above template to check every possible combination of valid queen placements.

In [ ]:
def backtrack(candidate):
    if find_solution(candidate):
        return True                            # return True once we've found a solution

    for next_candidate in list_of_candidates:
        if is_valid(next_candidate):
            place(next_candidate)
            if backtrack(next_candidate):      # if we've already found a solution...
                return True                    # ...propagate True up to the top of the call stack
            remove(next_candidate)
		return False                           # return False if we get to the end without returning a True

### Examples

**[LC 77. Combinations](https://leetcode.com/problems/combinations/) [Medium]**

This problem gives us two integers `n` and `k` and asks us to return all possible combinations of `k` numbers from the range `[1, n]`. Backtracking is a great choice for solving this problem because we can treat this as a CSP. Our candidates will be arrays of unique integers in the range `[1, n]` and our constraint is that a valid candidate can have `k` such integers. Whenever we build a candidate that has `k` unique integers we save that candidate as part of our answer, backtrack, and continue the recursion.

In [ ]:
def combine(n: int, k: int) -> list[list[int]]:

    def backtrack(first: int, curr: list):  # first is the num we start with in [1,n], curr is the current candidate represented as a list
        if len(curr) == k:                  # base case, our current candidate has k ints, store a copy of it
            combos.append(curr.copy())
            return                          # stop recursing deeper since we already have k ints
        
        for i in range(first, n+1):         # iterate through all remaining valid integers to add to our candidate
            curr.append(i)
            backtrack(i+1, curr)            # once we've added an integer, recurse with the set of integers larger than the added integer
            curr.pop()

    combos = []                             # store all valid candidates that we build during backtrack() that satisfy the CSP
    backtrack(1, [])                        # start at 1, and start with an empty candidate
    return combos

**[LC 79. Word Search](https://leetcode.com/problems/word-search/) [Medium]**

This problem asks us to find a word in a 2-dimensional array containing letters. Unlike in traditional word search, the word can "snake" as a 4-directional path through the grid. Upon first glance, you may think you can use BFS or DFS to solve this problem. However these algorithms don't work because they rely on maintaing an append only `visited` set whenever you visit a new grid cell. In our case however, if we realize our current candidate will not be able to build the target word, we need to backtrack, and upon backtracking we need to remove cells that are being backtracked from the `visited` set so we may possibly use them in a later path. Thus we use the backtracking algorithm for this problem.

In [ ]:
def does_word_exist(board: list[list[str]], word: str) -> bool:
    
    def backtrack(curr, idx):
        if idx == len(word):               # base case, we've traversed the same count of letters as the length of the target word
            return True

        i, j = curr                        # the curr grid cell our recursion sits on
        for nei in [(i+1,j), (i-1,j), (i,j+1), (i,j-1)]:
            x, y = nei
            if 0 <= x < m and 0 <= y < n and word[idx] == board[x][y] and nei not in visited:  # check if neighbor cell is valid
                visited.add(nei)           # if valid neighbor cell, add to visited set
                if backtrack(nei, idx+1):  # continue recursion
                    return True
                visited.remove(nei)        # if not valid candidate, pop last visited cell from visited set

        return False
    
    m, n = len(board), len(board[0])       # extract board dimensions
    for i in range(m):                     # iterate through each cell in the board
        for j in range(n):
            if board[i][j] == word[0]:     # initialize backtracking algo whenever we encounter the first letter in target word
                visited = set([(i,j)])     # visited will be the container we build candidates in
                if backtrack((i,j), 1):    # propogate successful backtrack
                    return True
    return False

**Check if Winning Card Hand**

This is my favorite interview problem that I've recieved in a FAANG interview. The problem asks us whether or not a given hand of 14 cards (enumerated 1-9, assume no suits) is a winning hand. A winning hand is one that contains 4 sets of triplets, where each triplet is a set of 3 cards that either form a run (e.g. 1,2,3) or are the same value (e.g. 1,1,1), and the remaining two cards are a pair (e.g. 2,2). 

The tricky part to this problem is figuring out how to build up our candidate solution. Instead of checking card by card, lets check 3 cards at a time, that way at each recursive step we check if we have a triplet run or three-of-a-kind, if we don't have either in any number 1-9 then we know we've hit a deadend and that we need to backtrack. Our base case as a result is when we're left with two cards, we just need to check if those two cards are a pair. An efficient way of building our candidate is to use a dictionary object where the key is the card and the value is the count of that card remaining in our hand. Rather than build up our candidate (our hand), in this example we actually subtract from it. 

In [ ]:
from collections import Counter

def is_winning_hand(hand: list[int]) -> bool:

    def backtrack():
        if sum(hand.values()) == 2:                                  # base case, a pair is left
            if len([val for val in hand.values() if val > 0]) == 1:
                return True

        for x in range(1, 10):                                       # loop through possible card values 1-9
            for triplet in [{x:3}, {x:1, x+1:1, x+2:1}]:             # check each triplet, three-ofa-kind or run
                valid_triplet = True
                for card, cnt in triplet.items():                    # check if triplet can be subtracted from our hand
                    if cnt > hand.get(card, 0):
                        valid_triplet = False
                if valid_triplet:                                    # if we find a valid triplet, continue the backtracking
                    for card, cnt in triplet.items():                # remove the triple from our hand
                        hand[card] -= cnt
                    if backtrack():                                  # propagate a valid sol to the top of the recursion stack
                        return True
                    for card, cnt in triplet.items():                # if we need to backtrack, add the triplet back to our hand
                        hand[card] += cnt

        return False
    
    hand = Counter(hand)
    return backtrack()